# Calculate economic values for IHS data given Germany materials and historical prices

In [215]:
# Import packages
import math
import numpy as np
import pandas as pd

In [224]:
prices_path = "C:/Users\lukec\OneDrive - University of Cambridge\PhD\Data\IHS\Methodology\PEP Yearbook Price History.xlsm"

ihs_data_path = "C:/Users\lukec\OneDrive - University of Cambridge\PhD\C-THRU\data_distribution\Methanol/"

manual_conversion_location = "C:/Users\lukec\OneDrive - University of Cambridge\PhD\Data\IHS\Methodology\manual_location_conversions.csv"

output_mats = ihs_data_path+'methanol_production_material_costs.csv'
output_prods = ihs_data_path+'methanol_production_plant_costs.csv'

utility_quarter = 'Q4 2021'
germany_loc_factor = 0.98
regions = ['USGC', 'China']

In [220]:
prices_file = pd.ExcelFile(prices_path)
german_prices = prices_file.parse('Germany metric')[['Unnamed: 0', utility_quarter]]

mat_columns = ['Code', 'Data Version', 'Source', 'Target', 'Research Year',
       'Geography', 'Plant capacity', 'Capacity unit', 'Product', 'Value',
       'Value unit', 'Uncertainty', 'Provenance', 'Source type', 'Unit Cost',
       'Cost Unit', 'Source cost']
germany_materials = pd.read_csv(ihs_data_path+'materials.csv', index_col=0)[mat_columns]
germany_materials.head()

germany_prods = pd.read_csv(ihs_data_path+'products.csv', index_col=0)

manual_conversion = pd.read_csv(manual_conversion_location)

## Need to convert to output when not all items are in conversion table yet

In [221]:
final_mats, final_prods = germany_materials, germany_prods

for geog in regions:

       geog_conversions = prices_file.parse(geog+' metric')[['Unnamed: 0', utility_quarter]].rename(columns={'Q4 2021':geog})
       manual_geog = manual_conversion[['Unnamed: 0', geog]]
       geog_conversion_table = pd.concat((geog_conversions, manual_geog))

       ## -> Insert catch here if there are sources missing a conversion factor

       new_geog_mats = germany_materials.copy()
       new_geog_mats['Geography'] = geog
       new_geog_mats['Unit Cost'] = new_geog_mats['Source'].replace(dict(zip(geog_conversion_table['Unnamed: 0'], geog_conversion_table[geog]))).replace({'LF': np.NAN})
       geog_loc_factor = float(geog_conversion_table[geog_conversion_table['Unnamed: 0'] == 'Location factor'][geog].values)/germany_loc_factor

       tonne_conversions = [1 if i in ['KG', np.NAN] else 1/1000 for i in new_geog_mats['Cost Unit']]
       new_geog_mats['Source cost'] = [i*j if not math.isnan(j) else germany_materials.iloc[num]['Source cost']*geog_loc_factor for num, (i, j) in enumerate(zip(new_geog_mats['Unit Cost'].astype(float), new_geog_mats['Value'].astype(float)))]*np.array(tonne_conversions)

       final_mats = pd.concat((final_mats, new_geog_mats))

       new_geog_prods = germany_prods.copy()
       new_geog_prods['Geography'] = geog
       new_geog_prods['Variable Cost'] = new_geog_mats.groupby('Target').sum()['Source cost'].values

       factor_columns = ['Investment (MM US$)', 'Fixed Costs', 'Overhead + Tax, Ins.', 'Depreciation', 'G&A, Sales, Res.', 'ROI (15%)']
       new_geog_prods[factor_columns] = new_geog_prods[factor_columns]*geog_loc_factor
       new_geog_prods['Plant Cash Cost '] = new_geog_prods['Variable Cost']+new_geog_prods['Fixed Costs']+new_geog_prods['Overhead + Tax, Ins.']
       new_geog_prods['Plant Gate Costs '] = new_geog_prods['Plant Cash Cost ']+new_geog_prods['Depreciation']
       new_geog_prods['Production Cost '] = new_geog_prods['Plant Gate Costs ']+new_geog_prods['G&A, Sales, Res.']
       new_geog_prods['Product Value '] = new_geog_prods['Production Cost ']+new_geog_prods['ROI (15%)']
       new_geog_prods['Product Price '] = new_geog_prods['Name'].replace(dict(zip(geog_conversion_table['Unnamed: 0'], geog_conversion_table[geog])))

       final_prods = pd.concat((final_prods, new_geog_prods))

In [225]:
final_mats.to_csv(output_mats, index=False)
final_prods.to_csv(output_prods, index=False)

In [223]:
final_prods

,Code,Geography,Data Version,Research Year,Process,Unit,Base Capacity (MM unit/year),Investment (MM US$),Type,Name,...,Fixed Costs,"Overhead + Tax, Ins.",Plant Cash Cost,Depreciation,Plant Gate Costs,"G&A, Sales, Res.",Production Cost,ROI (15%),Product Value,Product Price
806,2M-2052,Germany,2022 Q1,2020,GREEN METHANOL PRODUCTION PROCESS,KG,52.013437,252.328563,Product,METHANOL,...,20.445948,18.398105,458.346129,48.512188,506.858317,30.506663,537.364980,72.768282,610.133262,43.840634
1037,2M-1568,Germany,2022 Q1,2014,METHANOL (MEGA SCALE) FROM SYNGAS VIA HALDOR T...,KG,1642.457972,924.801863,Product,METHANOL,...,1.088302,1.634968,110.680166,5.630597,116.310763,6.566140,122.876902,8.445895,131.322798,43.840634
1038,2M-1565,Germany,2022 Q1,2014,METHANOL (MEGA SCALE) FROM SYNGAS VIA JM/ICI/D...,KG,1642.457972,832.464167,Product,METHANOL,...,0.996456,1.485791,112.538140,5.068405,117.606545,6.589955,124.196500,7.602607,131.799107,43.840634
1039,2M-1566,Germany,2022 Q1,2014,METHANOL (MEGA SCALE) FROM SYNGAS VIA LURGI TE...,KG,1642.457972,955.174871,Product,METHANOL,...,1.133403,1.689993,114.721955,5.815521,120.537475,6.803198,127.340673,8.723281,136.063955,43.840634
1040,2M-1567,Germany,2022 Q1,2014,METHANOL (MEGA SCALE) FROM SYNGAS VIA TOYO ENG...,KG,1642.457972,973.623662,Product,METHANOL,...,1.323018,1.788304,117.943820,5.927845,123.871665,6.987549,130.859214,8.891768,139.750982,43.840634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,2M-1063,China,2022 Q1,2003,METHANOL ON MEGA SCALE,KG,3465.142426,890.998387,Product,METHANOL,...,0.683017,0.815743,36.840736,2.571318,39.412054,0.629784,40.041838,3.856977,43.898816,42.800000
1070,2M-1535,China,2022 Q1,2014,METHANOL PRODUCTION BY JOHNSON MATTHEY TWO-STA...,KG,1642.004379,634.414034,Product,METHANOL,...,1.098691,1.222667,46.736620,3.863656,50.600276,4.200085,54.800362,5.795484,60.595846,42.800000
1071,2M-1845,China,2022 Q1,2018,METHANOL PRODUCTION FROM COKING GAS,KG,199.988876,70.641009,Product,METHANOL,...,7.872467,6.253921,40.851826,3.532247,44.384073,1.978208,46.362281,5.298370,51.660652,42.800000
1072,2M-1427,China,2022 Q1,2011,METHANOL PRODUCTION VIA TOYO PROCESS,KG,1642.004379,568.082374,Product,METHANOL,...,1.064347,1.081706,45.950227,3.459689,49.409915,4.460210,53.870126,5.189533,59.059658,42.800000
